In [1]:
import pandas as pd
import gzip
import csv
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen

In [2]:
# Read the compressed TSV file using pandas
titles = pd.read_csv("C:/Users/bini/Downloads/title.basics.tsv.gz", compression="gzip", sep='\t')
ratings = pd.read_csv("C:/Users/bini/Downloads/title.ratings.tsv.gz", compression="gzip", sep='\t')

C:\Users\bini\AppData\Local\Temp\ipykernel_12800\3758832417.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  titles = pd.read_csv("C:/Users/bini/Downloads/title.basics.tsv.gz", compression="gzip", sep='\t')


In [3]:
data = pd.merge(titles, ratings, how = "inner", on = "tconst")

In [4]:
data["startYear"] = data["startYear"].replace("\\N", 0).astype("int")

In [8]:
data_reduced = data[(data["numVotes"] >= 50000) & (data["titleType"] == "movie") & (data["startYear"] >= 1970)] # &   & 

In [9]:
data_reduced.shape

(3877, 11)

In [10]:
def budget_and_gross(id):

    movie_url = f"https://www.imdb.com/title/{id}/"
    movie_response = requests.get(movie_url, headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36'})
    movie_soup = BeautifulSoup(movie_response.text, 'html.parser')
    soup = BeautifulSoup(str(movie_soup.find_all('section', {"cel_widget_id":"StaticFeature_BoxOffice"})), 'html.parser')
    try:
        budget_tag = soup.find('li', {'data-testid': 'title-boxoffice-budget'})
        budget_value = budget_tag.find('span', {'class': 'ipc-metadata-list-item__list-content-item'}).text.strip()
        budget_value = budget_value[:-12].replace(",", "")
    except Exception:
        budget_value = None
    try:
        gross_worldwide_tag = soup.find('li', {'data-testid': 'title-boxoffice-cumulativeworldwidegross'})
        gross_worldwide_value = gross_worldwide_tag.find('span', {'class': 'ipc-metadata-list-item__list-content-item'}).text.strip()
        gross_worldwide_value = int(gross_worldwide_value[1:].replace(",", ""))
    except Exception:
        gross_worldwide_value = None
    return([budget_value, gross_worldwide_value])

In [11]:
budget_and_gross("tt1187043")

['₹550000000', 60262836]

In [12]:
data_reduced[["budget", "gross"]] = data_reduced["tconst"].apply(budget_and_gross).apply(pd.Series)

C:\Users\bini\AppData\Local\Temp\ipykernel_12800\4041921803.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_reduced[["budget", "gross"]] = data_reduced["tconst"].apply(budget_and_gross).apply(pd.Series)
C:\Users\bini\AppData\Local\Temp\ipykernel_12800\4041921803.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_reduced[["budget", "gross"]] = data_reduced["tconst"].apply(budget_and_gross).apply(pd.Series)


In [13]:
data_reduced.to_csv("C:/Users/bini/Downloads/data_reduced_4k.csv", index=False)

In [14]:
data_reduced.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,budget,gross
17967,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,\N,118,"Comedy,Fantasy,Romance",6.4,87925,$48000000,76019048.0
43428,tt0065421,movie,The Aristocats,The AristoCats,0,1970,\N,78,"Adventure,Animation,Comedy",7.1,111758,$4000000,35459543.0
43877,tt0065938,movie,Kelly's Heroes,Kelly's Heroes,0,1970,\N,144,"Adventure,Comedy,War",7.6,52628,$4000000,5200000.0
43948,tt0066026,movie,M*A*S*H,M*A*S*H,0,1970,\N,116,"Comedy,Drama,War",7.4,75784,$3500000,81600000.0
44110,tt0066206,movie,Patton,Patton,0,1970,\N,172,"Biography,Drama,War",7.9,106476,$12000000,61749765.0


In [24]:
def release_date(id):
    try:
        movie_url = f"https://www.imdb.com/title/{id}/releaseinfo/?ref_=tt_dt_rdat"
        movie_response = requests.get(movie_url, headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36'})
        movie_soup = BeautifulSoup(movie_response.text, 'html.parser')
        soup = BeautifulSoup(str(movie_soup.find_all('section', {"class":"ipc-page-section ipc-page-section--base"})), 'html.parser')
        release_section = soup.find('li', {'class': 'ipc-inline-list__item'})
        release_date = release_section.find('span', {'class': 'ipc-metadata-list-item__list-content-item'})
        release_date = str(release_date)[78:-7]
    except Exception:
        release_date = None
    return release_date

In [16]:
release_date("tt15398776")

'July 11, 2023'

In [25]:
data_reduced[["release_date"]] = data_reduced["tconst"].apply(release_date).apply(pd.Series)

C:\Users\bini\AppData\Local\Temp\ipykernel_12800\2482239549.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_reduced[["release_date"]] = data_reduced["tconst"].apply(release_date).apply(pd.Series)


In [23]:
data_reduced

,budget
17967,$48000000
43428,$4000000
43877,$4000000
43948,$3500000
44110,$12000000
...,...
1363699,None
1363770,$26000000
1365637,$15000000
1366348,None


In [26]:
data_reduced.to_csv("C:/Users/bini/Downloads/data_reduced_4k_v2.csv", index=False)

In [32]:
data_reduced[data_reduced["budget"].apply(lambda x: True if str(x)[0] == "$" else False)]

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,budget,gross,release_date
17967,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,\N,118,"Comedy,Fantasy,Romance",6.4,87925,$48000000,76019048.0,"December 11, 2001"
43428,tt0065421,movie,The Aristocats,The AristoCats,0,1970,\N,78,"Adventure,Animation,Comedy",7.1,111758,$4000000,35459543.0,"December 11, 1970"
43877,tt0065938,movie,Kelly's Heroes,Kelly's Heroes,0,1970,\N,144,"Adventure,Comedy,War",7.6,52628,$4000000,5200000.0,"January 1, 1970"
43948,tt0066026,movie,M*A*S*H,M*A*S*H,0,1970,\N,116,"Comedy,Drama,War",7.4,75784,$3500000,81600000.0,"January 25, 1970"
44110,tt0066206,movie,Patton,Patton,0,1970,\N,172,"Biography,Drama,War",7.9,106476,$12000000,61749765.0,"February 4, 1970"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1363246,tt9770150,movie,Nomadland,Nomadland,0,2020,\N,107,Drama,7.3,175977,$5000000,39458207.0,"September 11, 2020"
1363549,tt9777666,movie,The Tomorrow War,The Tomorrow War,0,2021,\N,138,"Action,Adventure,Drama",6.6,220617,$200000000,14400000.0,"June 30, 2021"
1363770,tt9784798,movie,Judas and the Black Messiah,Judas and the Black Messiah,0,2021,\N,126,"Biography,Drama,History",7.4,88568,$26000000,7478009.0,"February 1, 2021"
1365637,tt9844522,movie,Escape Room: Tournament of Champions,Escape Room: Tournament of Champions,0,2021,\N,88,"Action,Adventure,Horror",5.7,56955,$15000000,65774490.0,"July 1, 2021"
